In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
from random import randint
import time
from faker import Faker
chromedriver = 'C:/Users/vales/Anaconda3/Lib/ChromeDriver/chromedriver.exe'
os.environ['webdriver.chrome.driver'] = chromedriver

In [24]:
user = 'jacob.petersen+admin@dms.md'
pword = '!Q2w3e4r'
url = 'https://roz.test.dms.md:8443/Accounts/Account/Login'

fake = Faker()

#Navigate and login to roz

def login():
    browser = webdriver.Chrome(chromedriver)
    browser.get(url)
    
    username_box = browser.find_element_by_xpath('//*[@id="UserName"]')
    username_box.send_keys(user)
    
    pword_box = browser.find_element_by_xpath('//*[@id="Password"]')
    pword_box.send_keys(pword)
    
    login_btn = browser.find_element_by_xpath('//*[@id="Login"]')
    login_btn.click()
    
    return browser
    
def add_user():
    browser = login()
    
    add_user_btn = browser.find_element_by_xpath('//button[@class="btn btn-default addUser"]')
    add_user_btn.click()
    
    return browser

def create_phone_num():
    success = 0
    while success < 1:
        phone = fake.phone_number()[2:].replace('-','').replace('.','')
        if len(phone) == 10:
            try:
                int(phone)
                if int(phone[0]) > 1:
                    success = 1            
            except:
                pass
    
    return phone

def check_validations(browser):
    spans = browser.find_elements_by_xpath('//span[@class="error"]')
    
    validation_errors = []
    for span in spans:
        if len(span.get_attribute('style')) < 1:
            validation_errors.append(span.text)
    
    return validation_errors

def enter_user_data():
    browser = add_user()
    
    time.sleep(2)
    
    user_roles = browser.find_elements_by_xpath('//*[@id="usertype"]/option')
    rand_user_role_num = randint(1, len(user_roles) - 1)
    user_roles[rand_user_role_num].click()
    user_role = user_roles[rand_user_role_num].text
    
    full_name = fake.name()
    first_name = full_name.split(' ')[0]
    last_name = full_name.split(' ')[1]
    email = 'jacob.petersen+{}@dms.md'.format(first_name)
    phone1 = create_phone_num()
    phone2 = create_phone_num()
    
    city = fake.city()
    state = fake.state()
    gender_ls = ['M', 'F']
    gender = gender_ls[randint(0,1)]
    
    first_name_box = browser.find_element_by_xpath('//*[@id="firstname"]')
    first_name_box.send_keys(first_name + Keys.TAB + last_name + Keys.TAB + gender + Keys.ENTER)
    
    phone_box = browser.find_element_by_xpath('//*[@id="primaryphone"]')
    phone_box.send_keys(phone1 + Keys.TAB + phone2 + Keys.TAB + email + Keys.TAB + city + Keys.TAB\
                       + state + Keys.TAB)
    
    provider_specialties = []
    if user_role == 'Provider':
        specialty_dd = browser.find_element_by_xpath('//button[@class="multiselect dropdown-toggle btn btn-default"]')
        specialty_dd.click()
        
        specialties = browser.find_elements_by_xpath('//*[@class="multiselect-container dropdown-menu"]/li')
        
        for i in range(randint(0,5)):
            rand_specialty = randint(0, len(specialties) - 1)
            temp_specialty = specialties[rand_specialty]
            temp_specialty.click()
            provider_specialties.append(temp_specialty.text)
    
        specialty_dd.send_keys(Keys.TAB)
        
    create_btn = browser.find_element_by_xpath('//button[@class="btn btn-success savePersonal"]')
    create_btn.click()
    
    time.sleep(2)
    
    validation_errors = check_validations(browser)
    
    user_dict = {'user_role': user_role, 'first_name': first_name, 'last_name': last_name,
                'email': email, 'phone1': phone1, 'phone2': phone2, 'city': city, 'state': state,
                'gender': gender, 'validations': validation_errors, 'specialties': provider_specialties}
    
    return browser, user_dict


def verify_creation():
    browser, user_dict = enter_user_data()
    time.sleep(2)
    
    current_url = browser.current_url
    
    if current_url == 'https://roz.test.dms.md:8443/Accounts/User/Index':
        success = 1
    else:
        success = 0
        
    user_dict['success'] = success
    
    browser.close()
    
    return user_dict


In [25]:
create_users = []
for i in range(5):
    create_users.append(verify_creation())
   

In [26]:
create_users

[{'city': 'New Michael',
  'email': 'jacob.petersen+Denise@dms.md',
  'first_name': 'Denise',
  'gender': 'M',
  'last_name': 'Charles',
  'phone1': '3028956246',
  'phone2': '2594652975',
  'specialties': [],
  'state': 'Colorado',
  'success': 1,
  'user_role': 'Order Fulfillment',
  'validations': []},
 {'city': 'Pamelastad',
  'email': 'jacob.petersen+Michael@dms.md',
  'first_name': 'Michael',
  'gender': 'F',
  'last_name': 'Schmidt',
  'phone1': '5164033649',
  'phone2': '6962995865',
  'specialties': [],
  'state': 'Washington',
  'success': 0,
  'user_role': 'System Administrator',
  'validations': ['Email Already Exists']},
 {'city': 'North Anthony',
  'email': 'jacob.petersen+Luis@dms.md',
  'first_name': 'Luis',
  'gender': 'M',
  'last_name': 'Byrd',
  'phone1': '2703920580',
  'phone2': '2812288158',
  'specialties': [],
  'state': 'Kentucky',
  'success': 1,
  'user_role': 'Order Fulfillment',
  'validations': []},
 {'city': 'Hughestown',
  'email': 'jacob.petersen+Brett